In [1]:
import pandas as pd
import statsmodels.api as sm
import pymc as pm
import arviz as az
import numpy as np

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
df = pd.read_csv("../data/findings/new_orleans_pd_uof_2016_2022.csv")

# Drop rows with missing values in the columns of interest
df = df.dropna(subset=['use_of_force_effective', 'citizen_influencing_factors'])

df['use_of_force_effective'] = df['use_of_force_effective'].map({'yes': 1, 'no': 0})

df['mentally_unstable'] = df['citizen_influencing_factors'].apply(lambda x: 1 if 'mentally unstable' in x else 0)
df = df.dropna(subset=['use_of_force_effective'])
df

,tracking_id,originating_bureau,division_level,division,unit,working_status,shift_time,investigation_status,disposition,service_type,...,citizen_influencing_factors,citizen_distance_from_officer,citizen_age,citizen_build,citizen_height,citizen_arrested,citizen_arrest_charges,agency_y,citizen_uid,mentally_unstable
1,888fc1d84fd0bb20c52627a603025e24,field operations bureau,2nd district,c platoon,NaN,NaN,NaN,Completed,use of force authorized,arresting,...,alchohol and unknown drugs,0 feet to 1 feet,29.0,large,> 6'3'',yes,NaN,new-orleans-pd,354b7df12d7748042d9127690dfc73df,0
2,888fc1d84fd0bb20c52627a603025e24,field operations bureau,2nd district,c platoon,NaN,NaN,NaN,Completed,use of force authorized,arresting,...,alchohol and unknown drugs,0 feet to 1 feet,29.0,large,> 6'3'',yes,NaN,new-orleans-pd,354b7df12d7748042d9127690dfc73df,0
3,fc500256497f7b93fb89605cacec4eb8,field operations bureau,1st district,3rd platoon,NaN,NaN,NaN,Completed,use of force authorized,call for service,...,alchohol and unknown drugs,7 feet to 10 feet,29.0,medium,5'10'' to 6'0'',no,NaN,new-orleans-pd,934d6ce00d0375d9f08caebe6c2e67d9,0
4,731920c88a49cc3ec37ed7f995f28586,investigations and support bureau,criminal investigations,homicide,squad e,NaN,NaN,Completed,use of force authorized,call for service,...,unknown,0 feet to 1 feet,19.0,medium,5'7'' to 5'9'',yes,NaN,new-orleans-pd,ac04a4772e6a64b901112801421cd1c1,0
5,215d5ce2caf07ce5f8b2aa2abe372507,field operations bureau,8th district,c platoon,patrol,NaN,NaN,Completed,use of force authorized,arresting,...,unknown,0 feet to 1 feet,22.0,small,5'4'' to 5'6'',yes,illegal carrying of a weapon,new-orleans-pd,7f8031a29da0cd375373d6b2c1e0d6b1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7936,50cf4213bb9f9d2779eb96732dddcafc,NaN,NaN,day watch,day watch,unknown working status,NaN,NaN,use of force justified,NaN,...,theft,11 feet to 14 feet,NaN,medium,6'1'' to 6'3'',yes,NaN,new-orleans-pd,22ae7236c58de9ff16b2a1c292414f6f,0
7937,eccd5d053023e22017b251addf7b10ca,NaN,NaN,night watch,night watch,unknown working status,NaN,NaN,use of force justified,NaN,...,trespassing,0 feet to 1 feet,NaN,medium,5'4'' to 5'6'',yes,NaN,new-orleans-pd,9258b614461ed966fe0c74d5e8ceb43f,0
7938,eccd5d053023e22017b251addf7b10ca,NaN,NaN,night watch,night watch,unknown working status,NaN,NaN,use of force justified,NaN,...,disturbing the peace,0 feet to 1 feet,NaN,medium,5'4'' to 5'6'',yes,NaN,new-orleans-pd,c0a98a596a746690c7ab69bfe3806936,0
7939,eccd5d053023e22017b251addf7b10ca,NaN,NaN,night watch,night watch,unknown working status,NaN,NaN,use of force justified,NaN,...,public intoxication,0 feet to 1 feet,NaN,medium,5'4'' to 5'6'',yes,NaN,new-orleans-pd,78220e3eaf97d5a225947a137cfea918,0


In [3]:
X = sm.add_constant(df['mentally_unstable'])
y = df['use_of_force_effective']

# Fit the model
logit_model = sm.Logit(y, X)
result = logit_model.fit()

print(result.summary())

# """
# The coefficient for mentally_unstable is -0.2096. 
# This indicates that being mentally unstable is associated with a decrease in the log-odds of the use of force being effective
#, holding all other variables constant.

# The p-value for mentally_unstable is 0.044, which is less than the common significance level of 0.05. 
# This suggests that the effect of being mentally unstable on the effectiveness of the use of force is statistically significant.
# """

Optimization terminated successfully.
         Current function value: 0.381155
         Iterations 6
                             Logit Regression Results                             
Dep. Variable:     use_of_force_effective   No. Observations:                 7441
Model:                              Logit   Df Residuals:                     7439
Method:                               MLE   Df Model:                            1
Date:                    Wed, 12 Jul 2023   Pseudo R-squ.:               0.0006912
Time:                            11:19:46   Log-Likelihood:                -2836.2
converged:                           True   LL-Null:                       -2838.1
Covariance Type:                nonrobust   LLR p-value:                   0.04762
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.9494      0.037     52.323      0.000 

In [4]:
from scipy.stats import chi2_contingency

# Create a contingency table
contingency_table = pd.crosstab(df['use_of_force_effective'], df['mentally_unstable'])

# Perform the chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

chi2, p

# """
# The chi-square statistic is approximately 3.86.
# The p-value is approximately 0.049, which is less than the common significance level of 0.05.
# The p-value is less than 0.05, so we reject the null hypothesis that use_of_force_effective
# and mentally_unstable are independent. 
# This suggests that there is a statistically significant relationship between these two variables.
# """

(3.864392672215673, 0.04932104085460102)

In [5]:
with pm.Model() as model:
    # Priors for unknown model parameters
    beta_0 = pm.Normal('beta_0', mu=0, sigma=1)
    beta_1 = pm.Normal('beta_1', mu=0, sigma=1)

    # Expected value of outcome (use logistic link function)
    p = pm.math.invlogit(beta_0 + beta_1 * df['mentally_unstable'])

    # Likelihood
    y_obs = pm.Bernoulli('y_obs', p=p, observed=df['use_of_force_effective'])

    trace = pm.sample(2000, tune=1000, cores=1)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [beta_0, beta_1]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 19 seconds.


In [6]:
az.summary(trace)

# """
# beta_0 is called the intercept, and it represents the predicted outcome when all your predictor variables are zero. 
# In this case, it represents the "baseline" scenario where mentally_unstable is 0, meaning the citizen is not mentally unstable.

# The mean value of beta_0 is 1.946. In the context of logistic regression, this value is in terms of log-odds. 
# Convert it to odds by taking the exponential, exp(1.947), which gives us about 7. 
# The odds are a ratio, and in this case, it means that the odds of the use of force being effective 
# are 7 to 1 when the citizen is not mentally unstable.
# The 3% and 97% Highest Density Interval (HDI) is a range of values that likely contain the true value of beta_0. 
# In other words, we are 94% confident that the true value of beta_0 (if we had all possible data) 
# would fall between 1.880 and 2.020.

# beta_1 (Coefficient for mentally_unstable): The mean value is -0.202. 
# This is the change in the log odds of the 'use of force' being effective 
# for each unit increase in the 'mentally unstable' variable. 
# So, if a citizen is mentally unstable, the log odds of the 'use of force' being effective decrease by 0.204. 
# The 3% and 97% HDI values (-0.396 and -0.003) provide a credible interval for this parameter.
# """

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,1.948,0.036,1.881,2.019,0.001,0.000,3505.0,2948.0,1.0
beta_1,-0.201,0.103,-0.391,-0.007,0.002,0.001,2948.0,2532.0,1.0


In [7]:
df['is_summer'] = df['uof_occur_month'].apply(lambda x: 1 if x in [6, 7, 8, 9] else 0)

X = sm.add_constant(df['is_summer'])
y = df['use_of_force_effective']

logit_model = sm.Logit(y, X)
result = logit_model.fit()

print(result.summary())

# const (Intercept): The mean value is 1.9727. The intercept represents the log-odds of the 'use of force' being effective
# when the 'is_summer' variable is 0, i.e., not in the summer. Taking the exponential of this value, exp(1.9727), 
# we get the odds ratio, which is approximately 7.19. So, the odds of use of force being effective when it's not summer 
# are about 7 to 1.

# is_summer: The coefficient for is_summer is -0.1475. 
# This is the change in the log odds of the 'use of force' being effective for each unit increase in the is_summer variable. 
# So, when an incident occurs in summer, the log odds of the 'use of force' being effective decrease by 0.1475.

# The p-value for is_summer is 0.044, which is less than the common significance level of 0.05. 
# This suggests that the effect of an incident occurring in summer on the effectiveness of the use of force is statistically 
# significant.

Optimization terminated successfully.
         Current function value: 0.381150
         Iterations 6
                             Logit Regression Results                             
Dep. Variable:     use_of_force_effective   No. Observations:                 7441
Model:                              Logit   Df Residuals:                     7439
Method:                               MLE   Df Model:                            1
Date:                    Wed, 12 Jul 2023   Pseudo R-squ.:               0.0007045
Time:                            11:20:16   Log-Likelihood:                -2836.1
converged:                           True   LL-Null:                       -2838.1
Covariance Type:                nonrobust   LLR p-value:                   0.04552
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9727      0.043     46.024      0.000       1.889       2.0

In [8]:
contingency_table = pd.crosstab(df['use_of_force_effective'], df['is_summer'])
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("chi2:", chi2, "p-value:", p)

# The chi-square statistic is approximately 3.9.
# The p-value is approximately 0.048, which is less than the common significance level of 0.05.
# The p-value is less than 0.05, so we reject the null hypothesis that use_of_force_effective and is_summer are independent. 
# This suggests that there is a statistically significant relationship between these two variables.

chi2: 3.901995656964375 p-value: 0.04822878621992337


In [9]:
with pm.Model() as model:

    beta_0 = pm.Normal('beta_0', mu=0, sigma=1)
    beta_1 = pm.Normal('beta_1', mu=0, sigma=1)

    # Expected value of outcome (use logistic link function)
    p = pm.math.invlogit(beta_0 + beta_1 * df['is_summer'])

    # Likelihood
    y_obs = pm.Bernoulli('y_obs', p=p, observed=df['use_of_force_effective'])

    trace = pm.sample(2000, tune=1000, cores=1)
    
az.summary(trace)

#beta_0 (Intercept): The mean value is 1.971, which is the log odds of the 'use of force' being effective
# when the 'is_summer' variable is 0 (i.e., when the incident is not in the summer). 
# The 3% and 97% Highest Density Interval (HDI) values (1.888 and 2.052) give us a credible interval for this parameter. 
# This interval is the range within which we believe the true parameter value lies with a certain level of confidence 
# (here, 94% confidence).

# beta_1 (Coefficient for is_summer): The mean value is -0.145. 
# This is the change in the log odds of the 'use of force' being effective for each unit increase in the 'is_summer' variable. 
# So, when an incident occurs in summer, the log odds of the 'use of force' being effective decrease by 0.145. 
# The 3% and 97% HDI values (-0.288 and -0.011) provide a credible interval for this parameter.

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [beta_0, beta_1]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 20 seconds.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,1.969,0.042,1.891,2.045,0.001,0.001,1992.0,2115.0,1.0
beta_1,-0.142,0.071,-0.271,-0.008,0.002,0.001,1947.0,2247.0,1.0


In [10]:
df['was_hospitalized'] = (df['citizen_hospitalized'] == 'yes').astype(int)

df['was_mentally_unstable'] = (df['citizen_influencing_factors'] == 'mentally unstable').astype(int)

df = df.dropna(subset=['was_hospitalized', 'was_mentally_unstable'])

In [11]:
y = df['was_mentally_unstable']

X = df['was_hospitalized']
X = sm.add_constant(X)  

model = sm.Logit(y, X)
result = model.fit()

result.summary()

# The coefficient for 'was_hospitalized' is approximately 1.7368. 
# This value is the log odds; exponentiating this value gives us the odds ratio of 1.7368
# which means the odds of being mentally unstable are about 5.68 times higher for citizens 
# who were hospitalized compared to those who were not, assuming all other factors are equal.
# The p-value for 'was_hospitalized' is very close to 0, 
# which indicates that the relationship between hospitalization and mental instability is statistically significant.

Optimization terminated successfully.
         Current function value: 0.317632
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                             Logit Regression Results                            
=================================================================================
Dep. Variable:     was_mentally_unstable   No. Observations:                 7441
Model:                             Logit   Df Residuals:                     7439
Method:                              MLE   Df Model:                            1
Date:                   Wed, 12 Jul 2023   Pseudo R-squ.:                 0.09681
Time:                           11:20:46   Log-Likelihood:                -2363.5
converged:                          True   LL-Null:                       -2616.8
Covariance Type:               nonrobust   LLR p-value:                3.317e-112
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -3.0518      0.072    -42.384      0.000      -3.193      -2.911
was_hospitalized     1.7514      0.085     20.657      0.000       1.585       1.918
====================================================================================
"""

In [12]:
contingency_table = pd.crosstab(df['was_hospitalized'], df['was_mentally_unstable'])

chi2, p, dof, expected = chi2_contingency(contingency_table)

chi2, p
# The p-value for 'was_hospitalized' is very close to 0, 
# which indicates that the relationship between hospitalization and mental instability is statistically significant.

(508.3402788423857, 1.4565439986401333e-112)

In [13]:
df = df.dropna(subset=['citizen_race', 'citizen_hospitalized'])

df['citizen_hospitalized'] = df['citizen_hospitalized'].apply(lambda x: 1 if x == 'yes' else 0)

df = pd.get_dummies(df, columns=['citizen_race'])

races = ['asian / pacific islander', 'black', 'hispanic', 'indian', 'white']

In [14]:
logit_results = {}

for race in races:
    y = df['citizen_hospitalized']
    X = sm.add_constant(df[f'citizen_race_{race}']) 
    model = sm.Logit(y, X)
    result = model.fit(disp=0) 
    logit_results[race] = result.summary()

logit_results

# Asian / Pacific Islander: The coefficient is approximately 0.189. This suggests that, all else being equal, 
# being Asian / Pacific Islander is associated with an increase in the log-odds of hospitalization.

# Black: The coefficient is approximately -0.428. This suggests that, all else being equal, 
# being Black is associated with a decrease in the log-odds of hospitalization.

# Hispanic: The coefficient is approximately -0.430. This suggests that, all else being equal, 
# being Hispanic is associated with a decrease in the log-odds of hospitalization.

# Indian: The coefficient is approximately 1.639. This suggests that, all else being equal, 
# being Indian is associated with an increase in the log-odds of hospitalization.

# White: The coefficient is approximately 0.524. This suggests that, all else being equal, 
# being White is associated with an increase in the log-odds of hospitalization.

{'asian / pacific islander': <class 'statsmodels.iolib.summary.Summary'>
 """
                             Logit Regression Results                            
 Dep. Variable:     citizen_hospitalized   No. Observations:                 7343
 Model:                            Logit   Df Residuals:                     7341
 Method:                             MLE   Df Model:                            1
 Date:                  Wed, 12 Jul 2023   Pseudo R-squ.:               3.174e-05
 Time:                          11:20:46   Log-Likelihood:                -4937.6
 converged:                         True   LL-Null:                       -4937.8
 Covariance Type:              nonrobust   LLR p-value:                    0.5756
                                             coef    std err          z      P>|z|      [0.025      0.975]
 ---------------------------------------------------------------------------------------------------------
 const                                    -0.4122   

In [15]:
chi2_results = {}

for race in races:
    contingency_table = pd.crosstab(df['citizen_hospitalized'], df[f'citizen_race_{race}'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    chi2_results[race] = {'Chi-square Statistic': chi2, 'p-value': p}

chi2_results

# Asian / Pacific Islander: The p-value is approximately 0.69, which is greater than 0.05. Therefore, we do not 
# reject the null hypothesis that hospitalization is independent of being Asian / Pacific Islander.

# Black: The p-value is extremely small (much less than 0.05). Therefore, we reject the null hypothesis 
# and conclude that there is a significant association between hospitalization and being Black.

# Hispanic: The p-value is approximately 0.023, which is less than 0.05. Therefore, we reject the null hypothesis 
# and conclude that there is a significant association between hospitalization and being Hispanic.

# Indian: The p-value is small (less than 0.05). Therefore, we reject the null hypothesis and conclude that 
# there is a significant association between hospitalization and being Indian.

# White: The p-value is extremely small (much less than 0.05). Therefore, we reject the null hypothesis 
# and conclude that there is a significant association between hospitalization and being White.

{'asian / pacific islander': {'Chi-square Statistic': 0.15400258581105772,
  'p-value': 0.6947393176714691},
 'black': {'Chi-square Statistic': 42.59630405001336,
  'p-value': 6.728640459530483e-11},
 'hispanic': {'Chi-square Statistic': 5.142953677499426,
  'p-value': 0.023340904172551115},
 'indian': {'Chi-square Statistic': 11.365736636141692,
  'p-value': 0.0007481140066438316},
 'white': {'Chi-square Statistic': 54.32404923776395,
  'p-value': 1.7000838588979772e-13}}

In [16]:
df = df.dropna(subset=['citizen_sex'])

df = pd.get_dummies(df, columns=['citizen_sex'])
genders = ['female', 'male']

In [17]:
logit_results_gender = {}

for gender in genders:
    y = df['citizen_hospitalized']
    X = sm.add_constant(df[f'citizen_sex_{gender}']) 
    model = sm.Logit(y, X)
    result = model.fit(disp=0) 
    logit_results_gender[gender] = result.summary()

logit_results_gender

# For female', the coefficient is 0.3671. This means that, all else being equal, being female is associated with an 
# increase in the log-odds of hospitalization by about 0.37 units. 
# Conversely, for 'male', the coefficient is -0.3671, meaning being male is associated with a decrease in the 
# log-odds of hospitalization by about 0.37 units.

{'female': <class 'statsmodels.iolib.summary.Summary'>
 """
                             Logit Regression Results                            
 Dep. Variable:     citizen_hospitalized   No. Observations:                 7332
 Model:                            Logit   Df Residuals:                     7330
 Method:                             MLE   Df Model:                            1
 Date:                  Wed, 12 Jul 2023   Pseudo R-squ.:                0.003283
 Time:                          11:20:47   Log-Likelihood:                -4913.1
 converged:                         True   LL-Null:                       -4929.3
 Covariance Type:              nonrobust   LLR p-value:                 1.279e-08
                          coef    std err          z      P>|z|      [0.025      0.975]
 --------------------------------------------------------------------------------------
 const                 -0.4663      0.026    -18.109      0.000      -0.517      -0.416
 citizen_sex_female 

In [18]:
odds_ratios = {}

for gender in genders:
    y = df['citizen_hospitalized']
    X = sm.add_constant(df[f'citizen_sex_{gender}'])  # adding a constant
    model = sm.Logit(y, X)
    result = model.fit(disp=0)  # disp=0 suppresses convergence messages
    logit_results_gender[gender] = {"result": result, "summary": result.summary()}

for gender, outputs in logit_results_gender.items():
    odds_ratios[gender] = np.exp(outputs["result"].params)

odds_ratios

# The odds of hospitalization for females are 1.44 times the odds of hospitalization for non-females, 
# holding all else constant. This is about a 44% increase in the odds of hospitalization.
# The odds of hospitalization for males are 0.69 times the odds of hospitalization for non-males, 
# holding all else constant. This is about a 31% decrease in the odds of hospitalization.

{'female': const                 0.627301
 citizen_sex_female    1.485731
 dtype: float64,
 'male': const               0.932000
 citizen_sex_male    0.673069
 dtype: float64}